### Stim Tableau Simulator
This notebook explores stim's tableau simulator. The rotated surface code is implemented and its stabilizers generated.
To double check the stabilizer, CHP circuits are generated. CHP stabilizers are compared with those of stim.

In [43]:
import stim
import numpy as np

Below is an example how to use the tableau simulator in stim. A circuit computing unitary $C$ is generated. The tableau reports for each qubit the effect of the circuit, given $Z_n$ and $X_n$ as starting point. The tableau contains therefore entries $CZ_nC^{\dagger}$ and $CX_nC^{\dagger}$

In [45]:
s = stim.TableauSimulator()

s.h(0)
t = s.current_inverse_tableau()
print(t**-1)

+-xz-
| ++
| ZX


In [46]:
# d = 3

# circuit = stim.Circuit.generated(
#     "surface_code:rotated_memory_z",
#     rounds=1,
#     distance=d
# )
# print(repr(circuit))

In [47]:
class RotatedSurfaceCode:
    """
    Class for keeping track of coordinates for data qubits and
    measurement qubits in the rotated surface code.
    Helper functions generate stim circuits, stim tableau simulator 
    circuits and CHP circuit code to help with experiments.
    """
    
    def __init__(self, d=5, rotated=True):
        self.d = d
        self.rotated = rotated
        self.data_coords = []
        self.x_measurement_coords = []
        self.z_measurement_coords = []
        
        # place data qubits on odd coords
        for x in range(d):
            for y in range(d):
                self.data_coords.append((2*x+1, 2*y+1))
        
        # place measurement qubits on even 
        # coords (with exceptins on boundaries)
        for x in range(d+1):
            for y in range(d+1):
                on_boundary_1 = (x == 0) or (x == d)
                on_boundary_2 = (y == 0) or (y == d)
                parity = (x % 2) != (y % 2)
                
                if on_boundary_1 and parity:
                    continue
                if on_boundary_2 and not parity:
                    continue
                if parity:
                    self.x_measurement_coords.append((2*x,2*y))
                else:
                    self.z_measurement_coords.append((2*x,2*y))
                    
        self.z_order = [ (1, 1), (1,-1), (-1,1), (-1,-1) ]
        self.x_order = [ (1, 1), (-1,1), (1,-1), (-1,-1) ]
        self.prepare_circuit_data()
    
    def coord_to_index(self,x,y):
        (x,y) = (x, y - x % 2)
        return int(x + y * (self.d + 0.5))
        
    def prepare_circuit_data(self):

        # create coords to index mapping
        p2q = {}
        for q in self.data_coords:
            p2q[q] = self.coord_to_index(*q)
        for q in self.x_measurement_coords:
            p2q[q] = self.coord_to_index(*q)
        for q in self.z_measurement_coords:
            p2q[q] = self.coord_to_index(*q)
            
        # generate inverse mapping
        q2p = {v: k for k, v in p2q.items()}
        
        self.data_qubits = []
        self.measurement_qubits = []
        self.x_measurement_qubits = []
        self.all_qubits = []
        
        for q in self.data_coords:
            self.data_qubits.append(p2q[q])
        for q in self.x_measurement_coords:
            self.measurement_qubits.append(p2q[q])
            self.x_measurement_qubits.append(p2q[q])
        for q in self.z_measurement_coords:
            self.measurement_qubits.append(p2q[q])
        self.all_qubits.append(self.data_qubits)
        self.all_qubits.append(self.measurement_qubits)
        self.all_qubits.sort()
        self.data_qubits.sort()
        self.measurement_qubits.sort()
        self.x_measurement_qubits.sort()
        
        # generate CNOT gate targets
        self.cnot_targets = []
        for k in range(4):
            self.cnot_targets.append([])
            for m in self.x_measurement_coords:
                data = (m[0] + self.x_order[k][0], m[1] + self.x_order[k][1])
                if data in p2q:
                    self.cnot_targets[k].append(p2q[m])
                    self.cnot_targets[k].append(p2q[data])
            for m in self.z_measurement_coords:
                data = (m[0] + self.z_order[k][0], m[1] + self.z_order[k][1])
                if data in p2q:
                    self.cnot_targets[k].append(p2q[data])
                    self.cnot_targets[k].append(p2q[m])
        
    def generate_stim_circuit(self):
        """
        Generates one rotated surface code cycle as stim circuit
        """
        circuit_cycle = stim.Circuit()
        circuit_cycle.append_operation("RZ", self.data_qubits)
        circuit_cycle.append_operation("R", self.measurement_qubits)
        circuit_cycle.append_operation("H", self.x_measurement_qubits)
        for targets in self.cnot_targets:
            circuit_cycle.append_operation("TICK")
            circuit_cycle.append_operation("CNOT", targets)
        circuit_cycle.append_operation("TICK")
        circuit_cycle.append_operation("H", self.x_measurement_qubits)
        circuit_cycle.append_operation("TICK")
        circuit_cycle.append_operation("MR", self.measurement_qubits)
        return circuit_cycle
    
    def generate_chp_circuit(self):
        """
        Generates string for CHP stabilizer simulator
        ! Double check RX state preparation !
        """
        stim_circuit = self.generate_stim_circuit()
        chp_circuit = ""
        # for line in stim_circuit:
        for i in range(len(stim_circuit)):
            current_line = str(stim_circuit[i])
            if "TICK" in current_line:
                continue
            if "H" in current_line:
                for op in current_line.split()[1:]:
                    chp_circuit += "h {}\n".format(op)
            if "CX" in current_line:
                ops = current_line.split()[1:]
                ops = [" ".join(ops[i:i+2]) for i in range(0, len(ops), 2)]
                for op in ops:
                    chp_circuit += "c {}\n".format(op)
            if "MR" in current_line:
                for op in current_line.split()[1:]:
                    chp_circuit += "m {}\n".format(op)
        return chp_circuit
    
    def generate_stim_tableau_circuit(self):
        """
        Generates circuit for stim's tableau simulator
        """
        # create circuit for tableau simulator
        s = stim.TableauSimulator()
        c = self.generate_stim_circuit()
        s.do_circuit(c)
        return s

We first generate the circuit for stim's tableau simulator and extract the tableau

In [48]:
rsf = RotatedSurfaceCode(d=3, rotated=True)
s = rsf.generate_stim_tableau_circuit()
t = s.current_inverse_tableau()
t = t**-1

Next, we print the stabilizers for the data qubits

In [49]:
 def print_data_qubits_stabilizers(tableau, data_qubits):
    enc = {0: "_", 1: "X", 2: "Y", 3: "Z"}
    # extract stabilizers for data qubits
    X = Z = ""
    for q in data_qubits:
        pauli = tableau.z_output(q)
        pauli_str = "+" if int(pauli.sign.real) > 0 else "-"
        for q in data_qubits:
            pauli_str += enc[pauli[q]]
        if "Z" in pauli_str:
            Z += pauli_str + "\n"
        else:
            X += pauli_str + "\n"
    print(X+Z)

print_data_qubits_stabilizers(t, rsf.data_qubits)

-XX_______
-_XX_XX___
+___XX_XX_
+_______XX
+ZZZ______
+ZZ_ZZ____
+ZZ___Z___
+___Z__Z__
+___Z___ZZ



Additionally we can print the complete tableau for the rotated surface code circuit.<br>
**Note**, that the first qubit is 0 and the last qubit is 26. Some of the qubits are not touched in the circuit (e.g. *qubit 0*).

In [50]:
print(t)

+-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-xz-
| ++ +- +- +- ++ ++ ++ ++ ++ ++ ++ +- ++ ++ ++ ++ ++ ++ ++ ++ ++ ++ ++ ++ ++ ++
| XZ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ ZX _X Z_ __ _Z __ __ __ _Z _Z __ _Z __ __ __ __ __ __ __ __ __ __ __ __ __
| __ X_ XZ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ _X _X ZX __ _Z __ __ __ _Z _Z _X _Z __ __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ __ XZ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ _X __ XZ __ __ __ __ __ _X __ _Z __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ __ __ __ XZ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ __ __ __ __ XZ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ __ __ __ __ __ ZX _Z _Z __ __ __ _Z _Z _X Z_ __ _Z __ __ __ __ __ __
| __ __ __ __ __ __ __ __ __ XZ X_ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __
| __ __ __ _X __ __ __ __ _X _Z XZ _X _

We are now interested in learning the effect of the correction made by the decoder (e.g. deepq) on the stabilizers.<br>
**TODO**: Not clear how this works with deepq?

In [55]:
# create a copy of the original tableau
v = s.copy()
# apply logical observable
observable_qubits = [1,8,15]
v.x(*observable_qubits)
# apply some "hidden" errors
# v.x(5)
# v.x(10)
# we assume that the decoder wants to "correct" qubit 19 (bottom right data qubit)
v.x(19)
# next, we measure the logical observable again (unphysical way)
# Z observables should be deterministic and not random
print(v.peek_observable_expectation(stim.PauliString("+_Z_Z_Z")))
print(v.peek_observable_expectation(stim.PauliString("+________Z_Z_Z")))
print(v.peek_observable_expectation(stim.PauliString("+_______________Z_Z_Z")))
# X observables should be random
print(v.peek_observable_expectation(stim.PauliString("+_X______X______X")))
print(v.peek_observable_expectation(stim.PauliString("+___X______X______X")))
print(v.peek_observable_expectation(stim.PauliString("+_____X______X______X")))

-1
-1
1
0
0
0


In [32]:
# extract stabilizers for data qubits
vt = v.current_inverse_tableau()
vt = vt**-1
print_data_qubits_stabilizers(vt, rsf.data_qubits)

+XX_______
-_XX_XX___
-___XX_XX_
+_______XX
-ZZZ______
+ZZ_ZZ____
-ZZ___Z___
+___Z__Z__
-___Z___ZZ

